In [3]:
# import required libs
import glob
import os

import tensorflow as tf
import tensorflow_data_validation as tfdv
print('TF version: {}'.format(tf.version.VERSION))
print('TFDV version: {}'.format(tfdv.version.__version__))

TF version: 2.12.0
TFDV version: 1.14.0


In [7]:
# Read artifact information from metadata store.
# import beam_dag_runner

from tfx.orchestration import metadata
from tfx.types import standard_artifacts
from ml_metadata.metadata_store.metadata_store import metadata_store_pb2

# NOTE: make sure to have forwarded to metadata grpc server in your development/prod server to connect to it.
# e.g. kubectl port-forward service/metadata-grpc-service 7000:8080 -n kubeflow
metadata_connection_config = metadata_store_pb2.MetadataStoreClientConfig()
metadata_connection_config.host = "localhost"
metadata_connection_config.port = 7000
with metadata.Metadata(metadata_connection_config) as store:
    stats_artifacts = store.get_artifacts_by_type(standard_artifacts.ExampleStatistics.TYPE_NAME)
    schema_artifacts = store.get_artifacts_by_type(standard_artifacts.Schema.TYPE_NAME)
    anomalies_artifacts = store.get_artifacts_by_type(standard_artifacts.ExampleAnomalies.TYPE_NAME)


In [8]:
filtered_stats = [x for x in stats_artifacts if "StatisticsGen" in x.uri]
filtered_schema = [x for x in schema_artifacts if "SchemaGen" in x.uri]


In [9]:
filtered_stats

[]

In [10]:
# configure output paths
# Exact paths to output artifacts can also be found on KFP Web UI if you are using kubeflow.
stats_path = filtered_stats[-1].uri
train_stats_file = os.path.join(stats_path, 'Split-train', 'FeatureStats.pb')
eval_stats_file = os.path.join(stats_path, 'Split-eval', 'FeatureStats.pb')
print("Train stats file:{}, Eval stats file:{}".format(
    train_stats_file, eval_stats_file))

schema_file = os.path.join(filtered_schema[-1].uri, 'schema.pbtxt')
print("Generated schame file:{}".format(schema_file))
anomalies_file = os.path.join(anomalies_artifacts[-1].uri, 'anomalies.pbtxt')
print("Generated anomalies file:{}".format(anomalies_file))

IndexError: list index out of range

In [ ]:
tfdv.load_stats_binary("/local/projects/pixelml/artifacts/tfx_pipeline_output/vqa/StatisticsGen/statistics/20/Split-eval/FeatureStats.pb")

In [ ]:
# load generated statistics from StatisticsGen
train_stats = tfdv.load_stats_binary(train_stats_file)
eval_stats = tfdv.load_stats_binary(eval_stats_file)
tfdv.visualize_statistics(lhs_statistics=train_stats, rhs_statistics=eval_stats,
                          lhs_name='TRAIN_DATASET', rhs_name="EVAL_DATASET")

In [ ]:
# load generated schema from SchemaGen
schema = tfdv.load_schema_text(schema_file)
tfdv.display_schema(schema=schema)

In [ ]:
# load data vaildation result from ExampleValidator
anomalies = tfdv.load_anomalies_text(anomalies_file)
tfdv.display_anomalies(anomalies)